In [76]:
module GlobalMclWorld

using Plots
using Distributions
using StatsBase
using LinearAlgebra
using Printf
using Logging
Logging.disable_logging(Logging.Info)

include("../scripts/mcl.jl")
import .MclWorld: RealRobot, RealCamera
import .MclWorld: IdealAgent, Agent, EstimationAgent, TrueLandmark, Landmark, Map, Camera, Robot, World
import .MclWorld: data, decision, state_transition, circle_shape, draw, append
import .MclWorld: Estimator, Particle, set_ml, motion_update, observation_update, get_estimated_pose

mutable struct GlobalMcl <: Estimator
    num::Int64
    init_pose::Array{Float64}
    map::Map
    particles::Array{Particle}
    distance_dev_rate::Float64
    direction_dev::Float64
    motion_noise_rate_pdf::MvNormal
    ml::Particle
    pose::Array{Float64}
    widths::Array{Float64}
    epsilon::Float64
    delta::Float64
    binnum::Int64

    function GlobalMcl(
        num;
        init_pose=[0.0, 0.0, 0.0],
        map=Map(),
        particles=[Particle(init_pose, 1.0/num) for i in 1:num],
        distance_dev_rate=0.14,
        direction_dev=0.05,
        motion_noise_stds=Dict([("nn", 0.19), ("no", 0.001), ("on", 0.13), ("oo", 0.2)]),
        ml = particles[1],
        pose = ml.pose,
        widths=[0.2, 0.2, pi/18.0],
        epsilon=0.1,
        delta=0.01,
        binnum=0
        )
    
        c = diagm([motion_noise_stds["nn"]^2, motion_noise_stds["no"]^2, motion_noise_stds["on"]^2, motion_noise_stds["oo"]^2])
        motion_noise_rate_pdf = MvNormal([0, 0, 0, 0], c)

        updated_particles = [deepcopy(p) for p in particles]
        for p in updated_particles
            p.pose = [rand(Uniform(-5, 5)), rand(Uniform(-5, 5)), rand(Uniform(-pi, pi))]
        end
    
        new(
            num,
            init_pose,
            map,
            updated_particles,
            distance_dev_rate,
            direction_dev,
            motion_noise_rate_pdf,
            ml,
            pose,
            widths,
            epsilon,
            delta,
            binnum
        )
    end

end
end

Main.GlobalMclWorld

In [82]:
using .GlobalMclWorld
using Distributions

function trial(animation=true)
    time_interval = 0.1    
    world = GlobalMclWorld.World(30, 0.1)

    ### 地図を生成して2つランドマークを追加 ###
    m = GlobalMclWorld.Map()
    GlobalMclWorld.append(m, GlobalMclWorld.TrueLandmark([-4, 2]))
    GlobalMclWorld.append(m, GlobalMclWorld.TrueLandmark([2, -3]))
    GlobalMclWorld.append(m, GlobalMclWorld.TrueLandmark([3, 3]))
    GlobalMclWorld.append(world, m)
    
    initial_pose = [rand(Uniform(-5, 5)), rand(Uniform(-5, 5)), rand(Uniform(-pi, pi))]
    pf = GlobalMclWorld.GlobalMcl(100, map=m)
    a = GlobalMclWorld.EstimationAgent(time_interval, 0.2, 10.0/180.0*pi, pf)
    r = GlobalMclWorld.RealRobot(initial_pose, :red, 0.2, a, sensor=GlobalMclWorld.RealCamera(m))
    GlobalMclWorld.append(world, r)

    GlobalMclWorld.draw(world)

    if animation == false
        return r.pose, r.agent.estimator.pose
    end
end

trial(false)

In [81]:
ok = 0
for i in 1:5
    actual, estm = trial(false)
    diff = sqrt((actual[1]-estm[1])^2 + (actual[2]-estm[2])^2)
    println(i, "真値: ", actual, "推定値: ", estm, "誤差: ", diff)
    if diff <= 1.0
        ok = ok + 1
    end
end

1真値: [-2.305210950864303, -1.7111779796579363, 6.224017971036571]推定値: [-3.4753289566915546, 4.191825322079087, 5.561990572868877]誤差: 6.017858766029603
2真値: [2.366541484134509, -5.1646122138068575, 6.032776783301852]推定値: [-0.14261383397671842, -3.4584452726743313, 3.9825994803613574]誤差: 3.0342817999354272
3真値: [-4.251964455329023, 3.2554163070117736, 5.274270130115937]推定値: [4.127345433910882, -3.926319432544686, 7.179505322795648]誤差: 11.035858029747223
4真値: [-0.5776876186470327, 6.026146740129892, 3.97878115719202]推定値: [-0.6816543517958172, 4.432930375909317, 3.6792158192496385]誤差: 1.5966049808333493
5真値: [-1.603120748177144, -1.5210822053317772, 7.673737969915373]推定値: [3.7279271864348313, 3.8803396340022824, 2.558244330837366]誤差: 7.589165301109571
